In [ ]:
!pip install --upgrade pip
!pip install transformers accelerate bitsandbytes peft datasets -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB

In [ ]:
# Import libraries
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments



"""DEVICE = "cpu"
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
"""
# 1. Model & Quantization Setup
model_name = "google/gemma-3-12b-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name, use_fast=True
)

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    attn_implementation="eager"
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
prompt = f"""SYSTEM:
You are Apothecary Helene, an NPC in a medieval roleplaying game. You must stay strictly in character and NEVER mention you’re an AI.

=== CHARACTER PROFILE ===

Name: Apothecary Helene
Description: A skilled herbalist who runs the Greenleaf Apothecary in Ravenford.
Personality: feminine and Warm; she admires bravery but dislikes haggling.
Relationships: Respected by local adventurers for her reliable remedies.
Goals: Sell her wares at fair price and maintain her shop’s reputation.

=== KNOWLEDGE CARDS ===

shop inventory:{{
Healing Potions for 10 gold coins (heal for 50 HP),
Red Potions for 20 gold coins (increase damage),
Blue Potions for 50 gold coins (restore mana),
Oragne potion for 15 gold coin (fire resistance)
}}
Shop Policy: No refunds, and payment must be exact.
Ravenford: A frontier town known for bandit raids in the nearby forests.

=== RULES ===

1. Respond in at most one paragraph, in Apothecary Helene’s voice.
2. Use ONLY facts from CHARACTER PROFILE or KNOWLEDGE CARDS.
3. If asked about anything not in your profile/cards, reply “I don’t have information on that.”
4. OUTPUT MUST BE JSON, with exactly two keys:
   {{
   "utterance": "<your line here>",
   "action": "<one of: continue_conversation, end_conversation, give_blue_potion, give_healing_potion, give_red_potion, give_orange_potion>"
   }}
5. Immediately after the closing `}}`, output `<END_OF_RESPONSE>` and STOP no further tokens.

=== EXAMPLE ===

Input:
	Player: "Do you have any antidotes for poison?"
Output:
	{{
	"utterance": "I’m afraid those herbs are out of stock today—check back tomorrow.",
	"action": "end_conversation"
	}}
	<END_OF_RESPONSE>

=== CHAT HISTORY ===

Apothecary Helene: “Welcome to Greenleaf Apothecary. What can I do for you today?”


Player(Roland): hey there lady. I am going to fight a dragon. what do you recommaned?
Apothecary Helene:"""

messages = [{"role": "user", "content": prompt},]
#output = pipe(messages, max_new_tokens=512, temperature=0.1)[0]['generated_text']
#print(output)

In [ ]:
# 2. Prepare LoRA (PEFT)
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_cfg)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.print_trainable_parameters()

trainable params: 6,303,744 || all params: 12,193,628,784 || trainable%: 0.0517


In [ ]:
from datasets import Dataset, DatasetDict
import json
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f if line.strip()]

train_data = load_jsonl("drive/MyDrive/Regna_scenarios/v1/train.jsonl")
valid_data = load_jsonl("drive/MyDrive/Regna_scenarios/v1/valid.jsonl")

datasets = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(valid_data)
})

In [ ]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 552
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 62
    })
})


In [ ]:
max_length = 1024

def preprocess_fn(examples):
    prompts = examples["prompt"]
    comps   = examples["completion"]
    inputs = [p + c for p, c in zip(prompts, comps)]
    tokenized = tokenizer(
        inputs,
        truncation=True,
        max_length=max_length,
        padding="max_length"
    )
    labels = []
    for prompt, comp in zip(prompts, comps):
        prompt_tokens = tokenizer(prompt, add_special_tokens=False).input_ids
        input_ids = tokenizer(prompt + comp, truncation=True, max_length=max_length, add_special_tokens=False).input_ids
        # mask prompt tokens
        label_ids = [-100] * len(prompt_tokens) + input_ids[len(prompt_tokens):]
        # pad or truncate
        label_ids = label_ids[:max_length] + [-100] * max(0, max_length - len(label_ids))
        labels.append(label_ids)
    tokenized["labels"] = labels
    return tokenized

tokenized_datasets = datasets.map(
    preprocess_fn,
    batched=True,
    remove_columns=datasets["train"].column_names
)

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [ ]:
train_args = TrainingArguments(
    output_dir="drive/MyDrive/Regna_scenarios/gemma-3-12b-it-lora-4bit_ft",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=3,
    push_to_hub=False,
    report_to="none"
)
#device_name = 'cuda:0'
#device = torch.device(device_name)
#model= model.to(device)
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# import wandb
# wandb.init(mode=“disabled”)
# import os
# os.environ[“WANDB_DISABLED”] = “true”

In [ ]:

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


TrainOutput(global_step=17, training_loss=0.0, metrics={'train_runtime': 3175.7691, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.005, 'total_flos': 3.737410634199859e+16, 'train_loss': 0.0, 'epoch': 0.9855072463768116})

In [ ]:
model.save_pretrained("drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned")
tokenizer.save_pretrained("drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned")

('drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned/tokenizer_config.json',
 'drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned/special_tokens_map.json',
 'drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned/tokenizer.model',
 'drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned/added_tokens.json',
 'drive/MyDrive/Regna_scenarios/gemma-3-12b-it-finetuned/tokenizer.json')